# Kernel Operations & Entanglement

This notebook explores the Kernel as a stateless transformation engine and demonstrates entanglement concepts.

**Key Concepts**:
- Pure morphisms (stateless transformations)
- Multi-level operations (basic sets, entanglement, network)
- Isomorphism detection
- Singularity and coherence

## 1. Kernel: Stateless Transformations

The Kernel provides pure operations - same input always produces same output.

In [1]:
from core import Kernel, HLLSet

# Create kernel
kernel = Kernel()

print("Kernel is stateless:")
print("  - No storage")
print("  - No history")
print("  - No CAS (Content-Addressable Storage)")
print("  - Pure transformations only")
print(f"\nKernel instance: {kernel}")

Kernel is stateless:
  - No storage
  - No history
  - No CAS (Content-Addressable Storage)
  - Pure transformations only

Kernel instance: <core.kernel.Kernel object at 0x7fe02964a060>


## 2. Level 1: Basic Set Operations

Core morphisms for HLLSet manipulation.

In [2]:
# Absorb: tokens → HLLSet
hll_a = kernel.absorb(['apple', 'banana', 'cherry'])
hll_b = kernel.absorb(['banana', 'cherry', 'date'])

print("Basic Operations:\n")
print(f"A: {hll_a}")
print(f"B: {hll_b}")

# Union: A ∪ B
union = kernel.union(hll_a, hll_b)
print(f"\nA ∪ B: {union}")

# Intersection: A ∩ B
intersection = kernel.intersection(hll_a, hll_b)
print(f"A ∩ B: {intersection}")

# Difference: A - B
difference = kernel.difference(hll_a, hll_b)
print(f"A - B: {difference}")

Basic Operations:

A: HLLSet(983bcf39..., |A|≈4.0, backend=C/Cython)
B: HLLSet(9b080fbe..., |A|≈4.0, backend=C/Cython)

A ∪ B: HLLSet(eddbc507..., |A|≈5.0, backend=C/Cython)
A ∩ B: HLLSet(ef34c2bc..., |A|≈3.0, backend=C/Cython)
A - B: HLLSet(cede032c..., |A|≈2.0, backend=C/Cython)


## 3. Similarity Measurements

Compute similarity between HLLSets.

In [3]:
# Jaccard similarity
jaccard = kernel.similarity(hll_a, hll_b)
print(f"Jaccard similarity: {jaccard:.2%}")
print(f"  |A ∩ B| / |A ∪ B|")

# Cosine similarity (not in base Kernel, but in HLLSet)
cosine = hll_a.cosine(hll_b)
print(f"\nCosine similarity: {cosine:.2%}")

# Create more similar sets
hll_x = kernel.absorb(['a', 'b', 'c'])
hll_y = kernel.absorb(['a', 'b', 'd'])
sim_xy = kernel.similarity(hll_x, hll_y)
print(f"\nX vs Y similarity: {sim_xy:.2%} (2 common / 4 total)")

Jaccard similarity: 60.00%
  |A ∩ B| / |A ∪ B|

Cosine similarity: 75.00%

X vs Y similarity: 40.00% (2 common / 4 total)


## 4. Level 2: Fold Operations

Aggregate multiple HLLSets via union or intersection.

In [4]:
# Multiple HLLSets
hll_list = [
    kernel.absorb(['a', 'b', 'c']),
    kernel.absorb(['b', 'c', 'd']),
    kernel.absorb(['c', 'd', 'e']),
    kernel.absorb(['d', 'e', 'f'])
]

print(f"Created {len(hll_list)} HLLSets\n")

# Fold union: combine all via union
union_all = kernel.fold_union(hll_list)
print(f"Union of all: {union_all}")
print(f"  Cardinality: {union_all.cardinality():.2f}")
print(f"  Expected: ~6 (a, b, c, d, e, f)")

# Fold intersection: common to all
intersection_all = kernel.fold_intersection(hll_list)
print(f"\nIntersection of all: {intersection_all}")
print(f"  Cardinality: {intersection_all.cardinality():.2f}")

Created 4 HLLSets

Union of all: HLLSet(f70d249e..., |A|≈8.0, backend=C/Cython)
  Cardinality: 8.00
  Expected: ~6 (a, b, c, d, e, f)

Intersection of all: HLLSet(1ceaf73d..., |A|≈0.0, backend=C/Cython)
  Cardinality: 0.00


## 5. Level 3: Isomorphism Detection

Find approximate isomorphisms (ε-isomorphisms) between HLLSets.

In [6]:
# Create two installations (different data, similar structure)
installation_1 = kernel.absorb([f'token_{i}' for i in range(100)])
installation_2 = kernel.absorb([f'token_{i}' for i in range(95, 205)])

print(f"Installation 1: {installation_1.short_name}..., |A|≈{installation_1.cardinality():.0f}")
print(f"Installation 2: {installation_2.short_name}..., |A|≈{installation_2.cardinality():.0f}")

# Find isomorphism
epsilon = 0.05  # 5% tolerance
morphism = kernel.find_isomorphism(installation_1, installation_2, epsilon=epsilon)

if morphism:
    print(f"\nFound ε-isomorphism (ε={epsilon}):")
    print(f"  Source: {morphism.source_hash}")
    print(f"  Target: {morphism.target_hash}")
    print(f"  Similarity: {morphism.similarity:.2%}")
    print(f"  Is isomorphic: {morphism.is_isomorphism}")
else:
    print(f"\nNo ε-isomorphism found (similarity too low)")

Installation 1: fbd460b3..., |A|≈106
Installation 2: f1abc9c2..., |A|≈109

Found ε-isomorphism (ε=0.05):
  Source: fbd460b3e13faf38184bce01a4d5b7b9481b9107
  Target: f1abc9c253e7e8e5b4c8d1434f34f89901f23964
  Similarity: 4.33%
  Is isomorphic: True


## 6. Entanglement Validation

Check if multiple HLLSets are mutually entangled.

In [7]:
# Create related installations
base_tokens = [f'common_{i}' for i in range(50)]

installations = [
    kernel.absorb(base_tokens + [f'unique_a_{i}' for i in range(10)]),
    kernel.absorb(base_tokens + [f'unique_b_{i}' for i in range(10)]),
    kernel.absorb(base_tokens + [f'unique_c_{i}' for i in range(10)])
]

print(f"Created {len(installations)} related installations")
for i, inst in enumerate(installations, 1):
    print(f"  Installation {i}: {inst.short_name}..., |A|≈{inst.cardinality():.0f}")

# Validate entanglement
is_entangled, coherence = kernel.validate_entanglement(installations, epsilon=0.05)

print(f"\nEntanglement validation:")
print(f"  Entangled: {is_entangled}")
print(f"  Coherence: {coherence:.2%}")
print(f"\n  (High coherence means installations share common structure)")

Created 3 related installations
  Installation 1: ca7b44e5..., |A|≈57
  Installation 2: 4fbca95c..., |A|≈58
  Installation 3: 6c460897..., |A|≈60

Entanglement validation:
  Entangled: True
  Coherence: 69.93%

  (High coherence means installations share common structure)


## 7. Reproduction with Mutation

Create a reproduction of an HLLSet with slight mutation.

In [8]:
# Original HLLSet
original = kernel.absorb([f'data_{i}' for i in range(50)])

print(f"Original: {original.short_name}..., |A|≈{original.cardinality():.0f}")

# Reproduce (with potential mutation)
mutation_rate = 0.1  # 10% mutation
reproduced = kernel.reproduce(original, mutation_rate=mutation_rate)

print(f"Reproduced: {reproduced.short_name}..., |A|≈{reproduced.cardinality():.0f}")

# Check similarity
sim = kernel.similarity(original, reproduced)
print(f"\nSimilarity: {sim:.2%}")
print(f"  (Mutation rate {mutation_rate:.0%} affects similarity)")

Original: 1d01dd0c..., |A|≈52
Reproduced: 1d01dd0c..., |A|≈52

Similarity: 100.00%
  (Mutation rate 10% affects similarity)


## 8. Commitment (Stabilization)

Stabilize an HLLSet (in this system, HLLSets are already immutable).

In [9]:
# HLLSet to commit
uncommitted = kernel.absorb(['pending', 'data', 'tokens'])

print(f"Uncommitted: {uncommitted.short_name}...")

# Commit (stabilize)
committed = kernel.commit(uncommitted)

print(f"Committed: {committed.short_name}...")
print(f"\nSame content: {uncommitted.name == committed.name}")
print("  (HLLSets are already immutable, commit is identity operation)")

Uncommitted: fd77d4b7...
Committed: fd77d4b7...

Same content: True
  (HLLSets are already immutable, commit is identity operation)


## 9. Network Coherence

Measure coherence across a network of HLLSets.

In [10]:
# Create network of installations
import numpy as np

network = []
num_installations = 5

# Shared base + unique additions
shared_base = [f'shared_{i}' for i in range(30)]

for i in range(num_installations):
    unique = [f'unique_{i}_{j}' for j in range(20)]
    installation = kernel.absorb(shared_base + unique)
    network.append(installation)

print(f"Created network with {len(network)} installations")

# Measure pairwise similarities
similarities = []
for i in range(len(network)):
    for j in range(i+1, len(network)):
        sim = kernel.similarity(network[i], network[j])
        similarities.append(sim)

avg_coherence = np.mean(similarities)
print(f"\nNetwork coherence: {avg_coherence:.2%}")
print(f"  Min similarity: {min(similarities):.2%}")
print(f"  Max similarity: {max(similarities):.2%}")
print(f"  Std deviation: {np.std(similarities):.2%}")

Created network with 5 installations

Network coherence: 42.64%
  Min similarity: 39.73%
  Max similarity: 45.31%
  Std deviation: 1.56%


## 10. Composability

Morphisms compose naturally - chaining operations.

In [11]:
# Start with tokens
tokens = ['initial', 'data', 'set']

# Chain of transformations
result = (
    kernel.absorb(tokens)
    .union(kernel.absorb(['additional', 'tokens']))
)

# Add more
result = kernel.union(result, kernel.absorb(['even', 'more']))

print(f"Final result: {result}")
print(f"Cardinality: {result.cardinality():.2f}")
print(f"\nMorphisms compose:")
print("  absorb → union → union")
print("  Each step returns new HLLSet")
print("  Pure functional composition")

Final result: HLLSet(a55e4d50..., |A|≈8.0, backend=C/Cython)
Cardinality: 8.00

Morphisms compose:
  absorb → union → union
  Each step returns new HLLSet
  Pure functional composition


## Summary

**Kernel Capabilities**:

**Level 1: Basic Operations**
- absorb, union, intersection, difference
- Pure, stateless transformations

**Level 2: Entanglement**
- find_isomorphism: Detect similar structures
- validate_entanglement: Check mutual coherence
- reproduce: Create variations
- commit: Stabilize (identity in this system)

**Level 3: Network**
- Fold operations: Aggregate multiple HLLSets
- Coherence measurement: Network similarity
- Composability: Chain operations

**Design Principles**:
- Stateless: No storage, no history
- Pure: Same input → same output
- Immutable: Operations return new instances
- Content-addressed: All outputs named by hash

**Next**: See `05_manifold_os.ipynb` for OS-level orchestration.